In [4]:
import cosmology
%pylab inline

Populating the interactive namespace from numpy and matplotlib


A module to create NFW lens Surface Density ($\Sigma$),

density contrast ($\delta$),

and lensing kappa ($\kappa$).

Based on the integral functions of a spherical NFW profile: 

Wright & Brainerd(2000, ApJ, 534, 34)

$$\rho_M(z)=\frac{\rho_c(0)\Omega_M(0)}{a^3}$$

$$\rho_c(z)=\rho_c(0)*E(z)^2$$

$$\Omega_M(z)=\frac{\Omega_M(0)}{E(z)^{2} a^3}$$

In [41]:
#important constant
C_LIGHT=2.99792458e8        # m/s
GNEWTON=6.67428e-11         # m^3/kg/s^2
KG_PER_SUN=1.98892e30       # kg/M_solar
M_PER_PARSEC=3.08568025e16  # m/pc
def four_pi_G_over_c_squared():
    # We want it return 4piG/c^2 in unit of Mpc/M_solar 
    # in unit of m/kg
    fourpiGoverc2 = 4.0*np.pi*GNEWTON/(C_LIGHT**2)
    # in unit of pc/M_solar 
    fourpiGoverc2 *= KG_PER_SUN/M_PER_PARSEC
    # in unit of Mpc/M_solar
    fourpiGoverc2 /= 1.e6
    return fourpiGoverc2   


class nfw_lens():
    """   
    Based on the integral functions of a spherical NFW profile: 
    Wright & Brainerd(2000, ApJ, 534, 34)
    
    @param mass         Mass defined using a spherical overdensity of 200 times the critical density
                        of the universe, in units of M_solar/h.
    @param conc         Concentration parameter, i.e., ratio of virial radius to NFW scale radius.
    @param redshift     Redshift of the halo.
    @param ra           ra of halo center  [arcsec].
    @param dec          dec of halo center [arcsec].
    @param omega_m      Omega_matter to pass to Cosmology constructor. [default: 0.3]
                        omega_lam is set to 1-omega_matter
    """
    def __init__(self,mass,conc,redshift,ra,dec,omega_m=0.3):
        self.cosmo=cosmology.Cosmo(h=1,omega_m=omega_m)
        self.M = float(mass)
        self.c = float(conc)
        # \delta_c in equation (2)
        self.delta_nfw  =200./3*self.c**3/(np.log(1+self.c)-(1.*self.c)/(1+self.c))
        self.z = float(redshift)
        # E(z)^{-1}
        self.ezInv=self.cosmo.Ez_inverse(self.z)
        # scale radius
        self.a = 1./(1.+self.z)
        self.DaLens=self.cosmo.Da(0.,self.z)
        # ra_l,dec_l
        self.ra=ra
        self.dec=dec
        
        # First we get the virial radius, which is defined for some spherical overdensity as
        # 3 M / [4 pi (r_vir)^3] = overdensity
        # Here we have overdensity = 200 * rhocrit, to determine R200 (angular distance). 
        # The factor of 1.63e-5 comes from the following set of prefactors: 
        # (3 / (4 pi * 200 * rhocrit))^(1/3), where rhocrit = 2.8e11 h^2 M_solar / Mpc^3.  
        # The mass in the equation below is in M_solar/h, 
        # which is how the final units are Mpc/h.
        R200 = 1.63e-5*(self.M*self.ezInv**2)**(1./3.) # in Mpc/h
        self.rs = R200/self.c

        # convert scale radius in arcsec
        dl = self.cosmo.Da(0.,self.z) # in Mpc/h
        scale = self.rs / dl
        arcsec2rad = np.pi/180./3600
        self.rs_arcsec = scale/arcsec2rad
        return
    
    
    def Sigma(self,ra_s,dec_s):
        """Calculate Surface Density (Sigma) of halo.
        @param ra_s       ra of sources [arcsec]
        @param dec_s      dec of sources [arcsec]
        """
        
        # convenience: call with single number
        assert isinstance(ra_s,np.ndarray)==isinstance(dec_s,np.ndarray)
        if (not isinstance(ra_s,np.ndarray)) and (not isinstance(dec_s,np.ndarray)):
            ra_sA=np.array([ra_s], dtype='float')
            dec_sA=np.array([dec_s], dtype='float')
            return self.Sigma(ra_sA,dec_sA)[0]
        assert len(ra_s)==len(dec_s),
            'input ra and dec for '
        
        x = ((ra_s - self.ra)**2 + (dec_s - self.dec)**2)**0.5/self.rs_arcsec
        out = np.zeros_like(x, dtype=float)

        # 3 cases: x < 1, x > 1, and |x-1| < 0.001
        mask = np.where(x < 0.999)[0]
        a = ((1 - x[mask])/(x[mask] + 1))**0.5
        out[mask] = 2/(x[mask]**2 - 1) * (1 - np.log((1+a)/(1-a)) / (1-x[mask]**2)**0.5)

        mask = np.where(x > 1.001)[0]
        a = ((x[mask] - 1)/(x[mask] + 1))**0.5
        out[mask] = 2/(x[mask]**2 - 1) * (1 - 2*np.arctan(a)/(x[mask]**2 - 1)**0.5)

        # the approximation below has a maximum fractional error of 7.4e-7
        mask = np.where((x >= 0.999) & (x <= 1.001))[0]
        out[mask] = (22./15. - 0.8*x[mask])
        
        # critical density 
        rho_c   =   2.7722e11  #in unit of M_solar / Mpc^3
        # average matter density within R200 at redshift z
        # \delta_c \times \rho_c in equation (1),(11)
        rho_s   =   rho_c / self.ezInv**2 *self.delta_nfw
        
        return out* self.rs * rho_s
        
    def lensKernel(self,z_s):
        """Lensing kernel of halo as function of source redshift.
        @param z_s        redshift of sources
        """
        
        # convenience: call with single number
        if not isinstance(z_s, np.ndarray):
            return self.lensKernel(np.array([z_s], dtype='float'))[0]
        # lensing weights: the only thing that depends on z_s
        # First mask the data with z<z_l
        k_s =   np.zeros(len(z_s))
        mask=   z_s>self.z
        k_s[mask] =   self.cosmo.Da(self.z,z_s[mask])*self.DaLens/self.cosmo.Da(0.,z_s[mask])*four_pi_G_over_c_squared()
        return k_s


In [63]:
def compareGS():
    import galsim
    halo=nfw_lens(mass=1e15,conc=0.1,redshift=0.3,ra=0.,dec=0.)
    kappa=halo.lensKernel(2.)*halo.Sigma(2000.,0.)
    pos_cl  =   galsim.PositionD(0.,0.)
    haloGS  =   galsim.nfw_halo.NFWHalo(mass= 1e15,
                conc=0.1, redshift= 0.3,
                halo_pos=pos_cl ,omega_m= 0.3,
                omega_lam= 0.7)
    kappaGS=haloGS.getConvergence(pos=(2000.,0.),z_s=np.ones(1)*2.,units = "arcsec")
    print(halo.rs_arcsec,haloGS.rs_arcsec)
    print(halo.c,haloGS.c)
    print(kappa,kappaGS)
    return

4719.459416589283 4710.696878758708
0.1 0.1
0.04021036656352888 0.040107236201899356
